# GPT-2를 사용한 텍스트 생성 예제

이 노트북은 Hugging Face의 Transformers 라이브러리를 사용하여 GPT-2 모델로 텍스트를 생성하는 방법을 보여줍니다.

## 1. 필요한 라이브러리 임포트

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

## 2. 유틸리티 함수 정의

토큰화 결과와 다음 토큰 예측 확률을 시각화하기 위한 함수들을 정의합니다.

In [2]:
def show_tokenization(inputs, tokenizer):
    """토큰화된 결과를 데이터프레임 형태로 보여주는 함수
    Args:
        inputs: 토큰화된 입력값
        tokenizer: 사용중인 토크나이저
    Returns:
        DataFrame: 토큰 ID와 해당하는 실제 토큰을 보여주는 데이터프레임
    """
    return pd.DataFrame(
        [(id, tokenizer.decode(id)) for id in inputs["input_ids"][0]],
        columns=["id", "token"],
    )

def show_next_token_choices(probabilities, tokenizer, top_n=5):
    """다음 토큰의 확률 분포를 보여주는 함수
    Args:
        probabilities: 각 토큰별 확률값
        tokenizer: 사용중인 토크나이저
        top_n: 상위 몇 개의 확률을 보여줄지 지정
    Returns:
        DataFrame: 상위 n개의 토큰과 그 확률을 보여주는 데이터프레임
    """
    return pd.DataFrame(
        [
            (id, tokenizer.decode(id), p.item())
            for id, p in enumerate(probabilities)
            if p.item()
        ],
        columns=["id", "token", "p"],
    ).sort_values("p", ascending=False)[:top_n]

## 3. 모델과 토크나이저 로드

GPT-2 모델과 토크나이저를 Hugging Face 모델 허브에서 로드합니다.

In [3]:
print("모델과 토크나이저를 로딩중...")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

모델과 토크나이저를 로딩중...


## 4. 텍스트 토큰화 및 다음 토큰 예측

입력 텍스트를 토큰화하고 다음에 올 토큰을 예측해보겠습니다.

In [8]:
# 초기 텍스트 설정
text = "What is the best way to learn about generative"
print("초기 텍스트:", text)

# 텍스트 토큰화
inputs = tokenizer(text, return_tensors="pt")

# 토큰화 결과 출력
print("\n토큰화 결과:")
print(show_tokenization(inputs, tokenizer))

초기 텍스트: What is the best way to learn about generative

토큰화 결과:
             id   token
0  tensor(2061)    What
1   tensor(318)      is
2   tensor(262)     the
3  tensor(1266)    best
4   tensor(835)     way
5   tensor(284)      to
6  tensor(2193)   learn
7   tensor(546)   about
8  tensor(1152)   gener
9   tensor(876)   ative


## 5. 다음 토큰 예측 확률 계산

In [9]:
print("다음 토큰 예측 중...")
with torch.no_grad():
    logits = model(**inputs).logits[:, -1, :]
    probabilities = torch.nn.functional.softmax(logits[0], dim=-1)

# 상위 5개 토큰 선택지 출력
print("\n다음 토큰 상위 5개 선택지:")
print(show_next_token_choices(probabilities, tokenizer))

# 가장 확률이 높은 토큰 선택 및 텍스트에 추가
next_token_id = torch.argmax(probabilities).item()
print(f"\n선택된 다음 토큰 ID: {next_token_id}")
print(f"선택된 다음 토큰: {tokenizer.decode(next_token_id)}")

다음 토큰 예측 중...

다음 토큰 상위 5개 선택지:
          id         token         p
4673    4673      learning  0.190161
8300    8300   programming  0.166931
17219  17219       biology  0.075311
4981    4981        models  0.055784
16113  16113    algorithms  0.043036

선택된 다음 토큰 ID: 4673
선택된 다음 토큰:  learning


## 6. 전체 문장 생성

이번에는 `generate` 메서드를 사용하여 더 긴 텍스트를 생성해보겠습니다.

In [11]:
# 새로운 초기 텍스트로 시작
initial_text = "What is the best way to learn about generative"
inputs = tokenizer(initial_text, return_tensors="pt")

# generate 메소드를 사용하여 텍스트 생성
output = model.generate(
    **inputs,
    max_length=100,
    pad_token_id=tokenizer.eos_token_id
)

# 생성된 텍스트 출력
generated_text = tokenizer.decode(output[0])
print("\n생성된 텍스트:")
print(generated_text)


생성된 텍스트:
What is the best way to learn about generative learning?

The best way to learn about generative learning is to learn about the generative learning process.

The generative learning process is a process that involves learning about the generative learning process.

The generative learning process is a process that involves learning about the generative learning process.

The generative learning process is a process that involves learning about the generative learning process.

The generative learning process
